In [38]:
from tensorflow.keras import applications as app

In [58]:
## resnet modelini içeri aktaralım
resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,)

In [59]:
import tensorflow as tf

In [60]:
from tensorflow.keras.models import Sequential

In [61]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Input, Dense

In [245]:
import cv2
import numpy as np
import math
import pandas as pd

In [62]:
model = Sequential()

In [63]:
model.add(resnet)

In [64]:
model.add(Dense(10))

In [65]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
Total params: 25,646,722
Trainable params: 25,593,602
Non-trainable params: 53,120
_________________________________________________________________


In [150]:
def get_im(im_path:str)->list:
    """Verilen imaj yolundan imajı okur ve resnet için uygun hale getirir.
    
    Args:
        im_path (str): İmajın yolu
        
    Returns:
        list: işlenmiş imajın liste hali
    
    """
    im = cv2.imread(im_path)
    im = cv2.resize(im,dsize=(224,224))
    im = np.reshape(im,newshape=(1,224,224,3))
    return im

In [151]:
def get_results(im):
    """liste halindeki imajı resnet modeline vererek tahmin edilmiş vektörel değerleri döndürür.
    
    Args:
        im(numpy array) :  imajın numpy array hali
        
    Returns:
        list: imajın tahmin edilmiş vektörü

    """
    res = model(im)
    return tuple(res[0].numpy().tolist())

In [272]:
def find_similarity(vectors):
    """Verilen iki vektörize edilmiş görselin arasındaki öklid mesafesini hesaplar ve benzerliğe dönüştürür
    
    Args:
        vectors(list(tuple)) : imaj vektörlerinin listesi 
    
    """
    for i in range(len(vectors)):
        for j in range(len(vectors)):
            yield i,j,1-math.dist(vectors[i],vectors[j])

In [283]:
def process(im_paths:list):
    """
        imaj yollarını alır ve işlemlere tabi tutarak aralarındaki matematiksel mesafeyi hesaplar
        
        Args:
            im_paths(list): imajların yol listesi
        
        Calculations:
            Tahmin için Resnet50 kullanılır,
            Mesafe hesaplaması için Öklid mesafesi kullanılır.
    """
    
   
    yazi_format = lambda text: text.split(".")[0]
    ozel_yazdir = lambda res: print(
        "görseller: ", 
        yazi_format(im_paths[res[0]]),
        "-",
        yazi_format( im_paths[res[1]])
        ,"   mesafe:",res[2])
    
    predicted_vectors = []
    
    for path in im_paths:
        im = get_im(path)
        predicted_vectors.append(get_results(im))

    calculated = find_similarity(predicted_vectors)

    
    
    matrix = np.ones(shape=(len(im_paths),len(im_paths)))
    
    for result in calculated:
        ozel_yazdir(result)
        matrix[result[0],result[1]] = result[2]
        
    df = pd.DataFrame(
        matrix,
        columns=map(yazi_format,im_paths),
        index=map(yazi_format,im_paths))
    print(df)
    


In [284]:
process(["image.jpg","image2.jpg","image3.jpg"])

görseller:  image - image    mesafe: 1.0
görseller:  image - image2    mesafe: 0.8862613628628682
görseller:  image - image3    mesafe: 0.9570321492085606
görseller:  image2 - image    mesafe: 0.8862613628628682
görseller:  image2 - image2    mesafe: 1.0
görseller:  image2 - image3    mesafe: 0.9142179872769192
görseller:  image3 - image    mesafe: 0.9570321492085606
görseller:  image3 - image2    mesafe: 0.9142179872769192
görseller:  image3 - image3    mesafe: 1.0
           image    image2    image3
image   1.000000  0.886261  0.957032
image2  0.886261  1.000000  0.914218
image3  0.957032  0.914218  1.000000
